# Imports

In [2]:
import sys
sys.path.append("../src")

# Librairies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# SKlearn
from sklearn.tree import DecisionTreeClassifier 
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import accuracy_score

# Matplotlib
import matplotlib.pyplot as plt

# Constants

In [3]:
train_path = "../data/selected_train.csv"
train_not_norm_path = "../data/train_not_normalized.csv"

# Loading data

In [4]:
take_normalized_data = True

In [5]:
def preprocessing_data(data):
    scaler = StandardScaler()
    scaler.fit(data)
    return scaler.transform(data)

In [6]:
if take_normalized_data:
    # Already preprocessed data
    df_data = pd.read_csv(train_path, sep=",")
    X = df_data[df_data.columns.difference(['label'])].values
    
else:
    # Load data not preprocessed and preprocess it
    df_data = pd.read_csv(train_not_norm_path, sep=",")
    X = df_data[df_data.columns.difference(['label'])].values
    X = preprocessing_data(X)

y = df_data["label"].values

# Testing Decision Trees

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
clf    = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Accuracy=", accuracy_score(y_test, y_pred))

Accuracy= 0.6666666666666666


In [16]:
list_K = list(range(2, 20))
loo = LeaveOneOut()

avg_acc = 0.0

for train_idx, test_idx in loo.split(X):
    # Split train test
    X_train = X[train_idx]
    X_test  = X[test_idx] 
    y_train = y[train_idx]
    y_test  = y[test_idx]

    # Clf KNN
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)

    # Make predictions
    y_preds = clf.predict(X_test)
    avg_acc += accuracy_score(y_test, y_preds)

avg_acc = avg_acc/29
print("Avg acc", avg_acc)

Avg acc 0.9310344827586207
